## Initialitation

In [1]:
from pycqed.init.QuDev_testbench_init import *

Data directory set to: E:\Control software\data


Could not import qutip
`units` is deprecated for the `Parameter` class, use `unit` instead. <qcodes.instrument.parameter.StandardParameter: ch1_m1_del at 208513624>
`units` is deprecated for the `Parameter` class, use `unit` instead. <qcodes.instrument.parameter.StandardParameter: ch1_m2_del at 208514464>
`units` is deprecated for the `Parameter` class, use `unit` instead. <qcodes.instrument.parameter.StandardParameter: ch2_m1_del at 208538256>


initializing SGS100A's
Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/103189, firmware:3.1.19.15-3.20.347.26) in 0.05s
Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/103188, firmware:3.1.19.15-3.20.347.26) in 0.04s
Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/103718, firmware:3.1.19.15-3.20.347.26) in 0.02s
initializing AWG5014


`units` is deprecated for the `Parameter` class, use `unit` instead. <qcodes.instrument.parameter.StandardParameter: ch2_m2_del at 208538984>
`units` is deprecated for the `Parameter` class, use `unit` instead. <qcodes.instrument.parameter.StandardParameter: ch3_m1_del at 208554696>
`units` is deprecated for the `Parameter` class, use `unit` instead. <qcodes.instrument.parameter.StandardParameter: ch3_m2_del at 208555312>
`units` is deprecated for the `Parameter` class, use `unit` instead. <qcodes.instrument.parameter.StandardParameter: ch4_m1_del at 208603680>
`units` is deprecated for the `Parameter` class, use `unit` instead. <qcodes.instrument.parameter.StandardParameter: ch4_m2_del at 208604408>


Connected to: TEKTRONIX AWG5014C (serial:B010154, firmware:SCPI:99.0 FW:4.6.0.7) in 0.14s
initializing UHFQC
autoDetect selected the device dev2204 for the measurement.
Uploading file to device...
File successfully uploaded
Initialized UHFQC dev2204 in 0.85s
initializing heterodynes
initializing qubit


e:\control software\qcodes\qcodes\plots\pyqtgraph.py:46: UserWarning: windowTitle argument has been changed to window_title. Please update your call to QtPlot
  warnings.warn("windowTitle argument has been changed to window_title. Please update your call to QtPlot")


configuring parameters


Error getting or interpreting *IDN?: ''
Error getting or interpreting *IDN?: ''


Ran initialization in 11.96s


## Find the resonator frequency

In [ ]:
heterodyne.RF_power(-40)
qb2.measure_heterodyne_spectroscopy(freqs=np.linspace(7.19e9,7.191e9,20), close_fig=True)
#qb2.find_resonator_frequency(freqs=np.linspace(8.10e9,8.12e9,50), close_fig=False)
#qb2.find_resonator_frequency(close_fig=False)
#print("f0 = {} Hz".format(qb2.f_RO_resonator()))
#plt.show()

In [ ]:
MA = ma.MeasurementAnalysis()
MA.sweep_points[np.argmin(MA.measured_values[0])]

In [2]:
qb2.f_RO(7190420992.0)
heterodyne.frequency(7190420992.0)

## Calibrate the acquisition delay (for LO_modulated_heterodyne)

In [ ]:
heterodyne.frequency(7.1795e9)
qubit.measure_homodyne_acqusition_delay(delays=[0,100e-9],analyze=True) # this is necessary because of a bug where the first 
                                                                         # datapoint might be garbled
#qubit.find_homodyne_acqusition_delay(close_fig=False)
#print("delay = {} ns".format(qubit.optimal_acquisition_delay()))
#plt.show()

## Qubit spectroscopy

In [ ]:
qb2.spec_pow(-40)
readout_RF.power(-40)
qb2.measure_spectroscopy(freqs=np.linspace(6.023e9, 6.0245e9, 50), pulsed=True)

In [ ]:
QSA = ma.Qubit_Spectroscopy_Analysis(label='qb2',show_guess=False)
QSA.fit_results[0].values['f0']

In [3]:
qb2.f_qubit(6023614771.193965)

## Sweep acquisition delay (for calibration only)

In [ ]:
qubit.prepare_for_pulsed_spec()

spec_pars = qubit.get_spec_pars()
RO_pars = qubit.get_RO_pars()

AWG.clock_freq(1e9)
heterodyne.RO_length(2.2e-6)
heterodyne.nr_averages(1024)
qubit.cw_source.frequency(5.98e9)
qubit.heterodyne.frequency(7.1695e9)
qubit.spec_pow(-50)
qubit.readout_RF.power(-20)
qubit.RO_acq_marker_delay(-5e-6)
qubit.RO_amp()
qubit.cw_source.on()

sq.Pulsed_spec_seq(spec_pars, RO_pars)

qubit.AWG.start()

MC.set_sweep_function(UHFQC.awgs_0_userregs_2)
MC.set_sweep_points(np.arange(0, 1400, 25))
demod_mode = 'single' if qubit.heterodyne.single_sideband_demod() else 'double'
MC.set_detector_function(det.Heterodyne_probe(
    qubit.heterodyne,
    trigger_separation=qubit.heterodyne.trigger_separation(),
    demod_mode=demod_mode))
MC.soft_avg(1)
MC.run(name='pulsed-spec' + qubit.msmt_suffix)

qubit.cw_source.off()

## Rabi experiment
Remember to set the switch on upconversion board 1 to correct position

In [27]:
qb2.measure_rabi(amps=np.linspace(0.3, 0.45, 31))

Uploading file to device...
File successfully uploaded
Starting measurement: Rabi-n1_qb2
Sweep function: Rabi
Detector function: UHFQC_integrated_average


No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using defa

Programming 'Rabi_sequence' (31 element(s)) 	  finished in 2.65 seconds.
 100% completed 	elapsed time: 19.0s 	time left: 0.0s


In [28]:
#RabiA = ma.Rabi_Analysis(label='qubit')
RabiPA = ma.Rabi_parabola_analysis(label='qb2')
print("mean({}, {}) = {}".format(
    RabiPA.fit_res[0].values['x0'], 
    RabiPA.fit_res[1].values['x0'], 
    (RabiPA.fit_res[0].values['x0'] + RabiPA.fit_res[1].values['x0'])/2))

mean(0.3686146697489343, 0.36885904610556886) = 0.3687368579272516


In [29]:
qb2.amp180(0.3686146697489343)

## Ramsey

In [7]:
artificial_detuning = 4e6
qb2.measure_ramsey(times=np.linspace(0,2e-6,50), artificial_detuning=artificial_detuning)

Uploading file to device...
File successfully uploaded
Starting measurement: Ramsey_qb2
Sweep function: Ramsey
Detector function: UHFQC_integrated_average


No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using defa

Programming 'Ramsey_sequence' (50 element(s)) 	  finished in 2.96 seconds.
 100% completed 	elapsed time: 28.1s 	time left: 0.0s


In [8]:
RamA = ma.Ramsey_Analysis(auto=True)

fitted_freq = RamA.fit_res.params['frequency'].value
print('Artificial detuning: {:.2e}'.format(artificial_detuning))
print('Fitted detuning: {:.2e}'.format(fitted_freq))
print('Actual detuning:{:.2e}'.format(fitted_freq-artificial_detuning))
qb2.f_qubit() - fitted_freq + artificial_detuning

Artificial detuning: 4.00e+06
Fitted detuning: 4.03e+06
Actual detuning:2.95e+04


6023585271.0026035

In [9]:
qb2.f_qubit(6023585271.0026035)

## Rabi 90 amplitude

In [26]:
qb2.measure_rabi_amp90(scales=np.linspace(0.4, 0.6, 31))

Uploading file to device...
File successfully uploaded
Starting measurement: Rabi_amp90_scales_n1_qb2
Sweep function: Rabi_amp90
Detector function: UHFQC_integrated_average


No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using defa

Programming 'Rabi_amp90_sequence' (31 element(s)) 	  finished in 3.87 seconds.
 100% completed 	elapsed time: 20.3s 	time left: 0.0s


In [11]:
Rabi90PA = ma.Rabi_parabola_analysis(label='qb2')
print("mean({}, {}) = {}".format(
    Rabi90PA.fit_res[0].values['x0'], 
    Rabi90PA.fit_res[1].values['x0'], 
    (Rabi90PA.fit_res[0].values['x0'] + Rabi90PA.fit_res[1].values['x0'])/2))

mean(0.5101951186083692, 0.5087377261523199) = 0.5094664223803446


In [12]:
qb2.amp90_scale(0.5094664223803446)

## T1 measurement

In [ ]:
qb2.measure_T1(times=np.linspace(0,40e-6,100))

In [ ]:
T1A = ma.T1_Analysis(auto=True, close_fig=True)
T1 = T1A.T1
T1

## Spin-echo experiment

In [ ]:
artificial_detuning = 4e6 #Hz
qb2.measure_echo(times=np.linspace(0,4e-6,100), artificial_detuning=artificial_detuning)

In [ ]:
SEA = ma.Ramsey_Analysis(auto=True, close_fig=True, label='Echo')
print("T2 = {}".format(SEA.fit_res.params['tau'].value))

## Single-shot readout calibration

In [25]:
from pycqed.measurement import composite_detector_functions as cdet

def measure_ssro(self, no_fits=False,
                 return_detector=False,
                 MC=None,
                 analyze=True,
                 close_fig=True,
                 verbose=True, optimized_weights=False, SSB=False,
                 one_weight_function_UHFQC=False,
                 multiplier=1, nr_shots=4095):
    self.prepare_for_timedomain()
    if MC is None:
        MC = self.MC
    d = cdet.SSRO_Fidelity_Detector_Tek(
        'SSRO'+self.msmt_suffix,
        analyze=analyze,
        raw=no_fits,
        MC=MC,
        AWG=self.AWG, 
        acquisition_instr=self.UHFQC,
        pulse_pars=self.get_drive_pars(), 
        RO_pars=self.get_RO_pars(), 
        IF=self.f_RO_mod(),
        weight_function_I=self.RO_acq_weight_function_I(),
        weight_function_Q=self.RO_acq_weight_function_Q(),
        nr_shots=nr_shots, 
        one_weight_function_UHFQC=one_weight_function_UHFQC,
        optimized_weights=optimized_weights,
        integration_length=self.RO_acq_integration_length(),
        close_fig=close_fig, 
        SSB=SSB, 
        multiplier=multiplier,
        nr_averages=self.RO_acq_averages())
    if return_detector:
        return d
    d.prepare()
    d.acquire_data_point()
    # if analyze:
    #     return ma.SSRO_Analysis(rotate=soft_rotate, label='SSRO'+self.msmt_suffix,
    #                             no_fits=no_fits, close_fig=close_fig)
    
measure_ssro(qb2, optimized_weights=True, one_weight_function_UHFQC=True)

Uploading file to device...
File successfully uploaded


No sequencer config detected, using default config from station
No sequencer config detected, using default config from station


Programming 'OffOn_sequence' (2 element(s)) 	  finished in 1.64 seconds.


No sequencer config detected, using default config from station
No sequencer config detected, using default config from station


Programming 'OffOn_sequence' (2 element(s)) 	  finished in 1.65 seconds.
Starting measurement: SSRO_qb2_1
Sweep function: OffOn
Detector function: UHFQC_integration_logging_det


No sequencer config detected, using default config from station
No sequencer config detected, using default config from station


Programming 'OffOn_sequence' (2 element(s)) 	  finished in 1.67 seconds.
 99% completed 	elapsed time: 4.3s 	time left: 0.0sfrac1 in 0: 0.0098
1489508801.2027464


In [19]:
qb2.RO_pulse_delay(0)

## AllXY experiment
Measures different pairs of I, X180, Y180, X90 and Y90 pulses.

In [ ]:
qb2.measure_allxy()

In [ ]:
AllXYA = ma.AllXY_Analysis()

## Randomized Benchmarking

In [ ]:
nr_cliffords=[2,4,8,16]
qb2.measure_randomized_benchmarking(nr_cliffords)

In [ ]:
RBA = ma.RandomizedBenchmarking_Analysis(T1=T1, pulse_delay=qb2.pulse_delay())

## Butterfly experiment (still needs testing)

In [ ]:
from pycqed.measurement import awg_sweep_functions as awg_swf
reload(sq)
sq.station = station

qubit.int_log_det = det.UHFQC_integration_logging_det(
    UHFQC=UHFQC, AWG=AWG, channels=[
        qubit.RO_acq_weight_function_I(),
        qubit.RO_acq_weight_function_Q()],
    integration_length=qubit.RO_acq_integration_length(),
    nr_shots=4092)

def measure_butterfly(self, return_detector=False,
                      MC=None,
                      initialize=False,
                      post_msmt_delay=2e-6, case=True):
    self.prepare_for_timedomain()
    if MC is None:
        MC = self.MC
    MC.set_sweep_function(awg_swf.Butterfly(
        pulse_pars=self.get_drive_pars(), RO_pars=self.get_RO_pars(),
        initialize=initialize, post_msmt_delay=post_msmt_delay))
    MC.set_detector_function(self.int_log_det)
    MC.run('Butterfly{}initialize_{}'.format(self.msmt_suffix, initialize))

measure_butterfly(qubit)

In [ ]:
reload(ma)
# first perform SSRO analysis to extract the optimal rotation angle
# theta
a = ma.SSRO_discrimination_analysis(
    label='Butterfly',
    current_threshold=None,
    close_fig=False,
    plot_2D_histograms=True)

# the, run it a second time to determin the optimum threshold along the
# rotated I axis
b = ma.SSRO_discrimination_analysis(
    label='Butterfly',
    current_threshold=None,
    close_fig=False,
    plot_2D_histograms=True, theta_in=-a.theta)

c = ma.butterfly_analysis(
    close_main_fig=False, initialize=initialize,
    theta_in=-a.theta,
    threshold=b.opt_I_threshold, digitize=True, case=True)

In [ ]:
plt.show()

In [ ]:
c.butterfly_coeffs

In [ ]:
plt.plot(a.measured_values[1])
plt.show()

## temp

In [ ]:
#UHFQC.qas_0_result_length(2)
AWG.start()
UHFQC.awgs_0_enable(1)

# why do we need this?
try:
    UHFQC.awgs_0_enable()
except:
    UHFQC.awgs_0_enable()

while UHFQC.awgs_0_enable() == 1:
    time.sleep(0.01)
data = ['', '']
data[0] = UHFQC.qas_0_result_data_0_wave()[0]['vector']
data[1] = UHFQC.qas_0_result_data_1_wave()[0]['vector']
AWG.stop()
data
#data[0][0]**2 + data[1][0]**2

In [ ]:
reload(ZI_UHFQC)
UHFQC.single_acquisition_initialize()
data = UHFQC.single_acquisition(2)
UHFQC.single_acquisition_finalize()
data

In [ ]:
qubit.cw_source.power(-40)
qubit.spec_pulse_length(20e-6) #s
qubit.spec_pulse_depletion_time(10e-6) #s
qubit.RO_pulse_length(2.2e-6) #s
qubit.RO_acq_marker_delay(0) #s
qubit.RO_pulse_delay(500e-9) #s
readout_RF.pulsemod_state('On')

In [ ]:
qubit.measure_spectroscopy(freqs=np.linspace(5.8e9, 6.2e9, 50), pulsed=True)

In [ ]:
qubit.readout_RF.close()
qubit.readout_LO.close()

In [ ]:
station.components.keys()

In [ ]:
AWG.clock_freq(1e9)

In [ ]:
readout_LO.power()

In [ ]:
heterodyne.nr_averages()

In [ ]:
qubit.RO_pulse_power()

In [ ]:
readout_RF.pulsemod_state("On")

In [ ]:
for instr in station.components.values():
    instr.close()